In [18]:
pwd


'/home/htc/arostami'

In [19]:
conda init 

no change     /opt/conda/condabin/conda
no change     /opt/conda/bin/conda
no change     /opt/conda/bin/conda-env
no change     /opt/conda/bin/activate
no change     /opt/conda/bin/deactivate
no change     /opt/conda/etc/profile.d/conda.sh
no change     /opt/conda/etc/fish/conf.d/conda.fish
no change     /opt/conda/shell/condabin/Conda.psm1
no change     /opt/conda/shell/condabin/conda-hook.ps1
no change     /opt/conda/lib/python3.11/site-packages/xontrib/conda.xsh
no change     /opt/conda/etc/profile.d/conda.csh
no change     /home/htc/arostami/.bashrc
No action taken.

Note: you may need to restart the kernel to use updated packages.


In [20]:
pip install numpy==1.23.5


Note: you may need to restart the kernel to use updated packages.


In [21]:
pip install openmm

Note: you may need to restart the kernel to use updated packages.


In [22]:
from openmm.app import *
from openmm import *
from openmm.unit import *
from sys import stdout

In [ ]:
import simtk.openmm as mm
import simtk.openmm.app as app
import simtk.unit as unit
from openmmforcefields.generators import SystemGenerator
from openmmtools.integrators import LangevinIntegrator

# Load the protein structure file (PDB format)
pdb = app.PDBFile('/data/numerik/ag_cmd/arostami/fbdd/8g68.pdb')

# Load Amber force field parameters (Amber ff99SB-ILDN)
forcefield = app.ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')

# Parameterize ligand using GAFF (use a tool like Antechamber to generate ligand parameters)
ligand_prmtop = app.AmberPrmtopFile('ligand.prmtop')
ligand_inpcrd = app.AmberInpcrdFile('ligand.inpcrd')

# Create the system from the protein, ligands, water, and ions
modeller = app.Modeller(pdb.topology, pdb.positions)

# Add the ligand to the system
modeller.add(ligand_prmtop.topology, ligand_inpcrd.positions)

# Add the solvent (90 x 90 x 90 A³ cubic box, 150 mM NaCl)
modeller.addSolvent(forcefield, model='tip3p', boxSize=unit.Quantity((90, 90, 90), unit.angstroms), ionicStrength=0.15 * unit.molar)

# Create the system using Amber ff99SB-ILDN force field
system = forcefield.createSystem(modeller.topology, nonbondedMethod=app.PME, constraints=app.HBonds)

# Apply the force field parameters to the ligands (use GAFF parameters here)
ligand_forcefield = app.ForceField('gaff.xml')
ligand_system = ligand_forcefield.createSystem(ligand_prmtop.topology)

# Combine the ligand system and protein system
for force in ligand_system.getForces():
    system.addForce(force)

# Define an integrator (Langevin dynamics with constant temperature)
integrator = LangevinIntegrator(temperature=300*unit.kelvin, collision_rate=1/unit.picoseconds, timestep=2.0*unit.femtoseconds)

# Add barostat to keep pressure constant (1 atm)
system.addForce(mm.MonteCarloBarostat(1*unit.atmosphere, 300*unit.kelvin))

# Set up the simulation with a platform (CUDA for GPUs, CPU otherwise)
platform = mm.Platform.getPlatformByName('CUDA') # Or use 'CPU' if CUDA is unavailable
simulation = app.Simulation(modeller.topology, system, integrator, platform)

# Set initial positions
simulation.context.setPositions(modeller.positions)

# Minimize the energy of the system
simulation.minimizeEnergy()

# Add a random seed to ensure proper starting velocities
simulation.context.setVelocitiesToTemperature(300*unit.kelvin)

# Run the simulation
simulation.reporters.append(app.StateDataReporter('output.log', 1000, step=True, potentialEnergy=True, temperature=True))
simulation.reporters.append(app.DCDReporter('trajectory.dcd', 1000))

# Run the simulation for 100 ns
simulation.step(50000000)


In [2]:
conda init

no change     /opt/conda/condabin/conda
no change     /opt/conda/bin/conda
no change     /opt/conda/bin/conda-env
no change     /opt/conda/bin/activate
no change     /opt/conda/bin/deactivate
no change     /opt/conda/etc/profile.d/conda.sh
no change     /opt/conda/etc/fish/conf.d/conda.fish
no change     /opt/conda/shell/condabin/Conda.psm1
no change     /opt/conda/shell/condabin/conda-hook.ps1
no change     /opt/conda/lib/python3.11/site-packages/xontrib/conda.xsh
no change     /opt/conda/etc/profile.d/conda.csh
no change     /home/htc/arostami/.bashrc
No action taken.

Note: you may need to restart the kernel to use updated packages.


In [2]:
!mamba install -y -c conda-forge openmmtools


Looking for: ['openmmtools']

warning  libmamba Cache file "/scratch/htc/arostami/conda/pkgs/cache/497deca9.json" was modified by another program
warning  libmamba Cache file "/scratch/htc/arostami/conda/pkgs/cache/09cdf8bf.json" was modified by another program
warning  libmamba Could not parse mod/etag header
[+] 0.0s
conda-forge/label/openff-interchange_rc/linux-64.. ━━━╸━━━━━━━━━━━   0.0 B  0.0s[+] 0.1s
conda-forge/label/openff-interchange_rc/linux-64.. ━━━╸━━━━━━━━━━━   0.0 B  0.1s[+] 0.2s
conda-forge/label/openff-interchange_rc/linux-64.. ━━━╸━━━━━━━━━━━   0.0 B  0.2s[+] 0.3s
conda-forge/label/openff-interchange_rc/linux-64.. ━━━━╸━━━━━━━━━━   0.0 B  0.3s[+] 0.4s
conda-forge/label/openff-interchange_rc/linux-64.. ━━━━╸━━━━━━━━━━   0.0 B  0.4s[+] 0.5s
conda-forge/label/openff-interchange_rc/linux-64.. ━━━━╸━━━━━━━━━━   0.0 B  0.5sconda-forge/label/openff-interchange_rc/linux-64..  0.6s
warning  libmamba Could not parse mod/etag header
[+] 0.0s
conda-forge/label/openff-interchange_

In [3]:
import openmm as mm
import openmm.app as app
import openmm.unit as unit
from openmmtools.integrators import LangevinIntegrator

# Load the protein and ligand PDB files
pdb = app.PDBFile('/data/numerik/ag_cmd/arostami/fbdd/8g68.pdb')  # Or load them separately and combine using Modeller

# Define force fields for protein and ligand
forcefield = app.ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')  # For protein
ligand_forcefield = app.ForceField('gaff.xml')  # For ligand, assuming ligand is already parameterized

# Create a system
modeller = app.Modeller(pdb.topology, pdb.positions)

# Add solvent with 150 mM NaCl in a cubic box of 90x90x90 Å³
modeller.addSolvent(forcefield, model='tip3p', boxSize=unit.Quantity((90, 90, 90), unit.angstroms), ionicStrength=0.15 * unit.molar)

# Generate the system with Amber ff99SB-ILDN force field
system = forcefield.createSystem(modeller.topology, nonbondedMethod=app.PME, constraints=app.HBonds)

# Add N-methyl cap to the C-terminus of the protein (optional, depending on the input PDB)
# You can modify the PDB beforehand or programmatically apply it here

# Apply restraints to heavy atoms during equilibration
for force in system.getForces():
    if isinstance(force, mm.HarmonicBondForce):
        force.setUsesPeriodicBoundaryConditions(True)

# Add dihedral restraints for substrate-binding loop (SBL)
# This requires defining specific dihedral angles in the PDB for restraint

# Equilibration with harmonic position restraints
integrator_eq = LangevinIntegrator(temperature=300*unit.kelvin, collision_rate=1/unit.picoseconds, timestep=2.0*unit.femtoseconds)
system.addForce(mm.MonteCarloBarostat(1*unit.atmosphere, 300*unit.kelvin))

# Create the simulation
simulation_eq = app.Simulation(modeller.topology, system, integrator_eq)
simulation_eq.context.setPositions(modeller.positions)

# Minimize energy
simulation_eq.minimizeEnergy()

# Apply position restraints to heavy atoms and taper them over the equilibration run
# Linearly reduce the strength of the restraints over 50 ns
for step in range(50):  # Assuming 50 steps, each representing 1 ns of equilibration
    restraint_force = 5.0 * (50 - step) / 50.0  # Taper from 5 kcal/mol/Å² to 0
    # Apply harmonic restraints to protein heavy atoms
    # Restraint application code here (custom based on specific atoms or atom groups)

    simulation_eq.step(1000)  # Run for 1 ns (modify based on actual timestep size)

# Switch to production run in NVT ensemble at 310K, using a RESPA integrator
integrator_prod = mm.RespaIntegrator(2.4 * unit.femtoseconds)

# Switch to NVT ensemble (constant volume and temperature)
simulation_prod = app.Simulation(modeller.topology, system, integrator_prod)
simulation_prod.context.setVelocitiesToTemperature(310 * unit.kelvin)

# Add long-range electrostatics (Gaussian split Ewald) calculated every 7.5 fs
# Set up electrostatics method in the force field or simulation

# Run the production simulation for 100 µs
steps_per_microsecond = int(1e6 / (2.4 * 1e-3))  # Number of steps per µs with 2.4 fs timestep
simulation_prod.reporters.append(app.StateDataReporter('/data/numerik/ag_cmd/arostami/fbdd/output.log', 1000, step=True, potentialEnergy=True, temperature=True))
simulation_prod.reporters.append(app.DCDReporter('/data/numerik/ag_cmd/arostami/fbdd/trajectory.dcd', 1000))
simulation_prod.step(steps_per_microsecond * 100)  # Run for 100 µs


Warning on use of the timeseries module: If the inherent timescales of the system are long compared to those being analyzed, this statistical inefficiency may be an underestimate.  The estimate presumes the use of many statistically independent samples.  Tests should be performed to assess whether this condition is satisfied.   Be cautious in the interpretation of the data.

****** PyMBAR will use 64-bit JAX! *******
* JAX is currently set to 32-bit bitsize *
* which is its default.                  *
*                                        *
* PyMBAR requires 64-bit mode and WILL   *
* enable JAX's 64-bit mode when called.  *
*                                        *
* This MAY cause problems with other     *
* Uses of JAX in the same code.          *
******************************************



ValueError: Could not locate file "gaff.xml"

In [ ]:
forcefield = ForceField('hmmm_lipid.xml', 'protein.xml')
system = forcefield.createSystem(pdb.topology, nonbondedMethod=PME, 
                                 nonbondedCutoff=1*nanometer, constraints=HBonds)
force = CustomExternalForce('k*(z^2)')
force.addGlobalParameter('k', 5.0*kilocalories_per_mole/angstrom**2)

for atom in pdb.topology.atoms():
    if atom.residue.name in ['POPC', 'CHL1']:  # Adjust for your specific lipid types
        force.addParticle(atom.index)

system.addForce(force)
integrator = LangevinIntegrator(300*kelvin, 1/picosecond, 0.002*picoseconds)
simulation = Simulation(pdb.topology, system, integrator)
simulation.context.setPositions(pdb.positions)
simulation.minimizeEnergy()
simulation.reporters.append(StateDataReporter(stdout, 1000, step=True, 
                                              potentialEnergy=True, temperature=True))
simulation.reporters.append(DCDReporter('trajectory.dcd', 1000))
simulation.step(100000)  # Run for 100,000 steps


In [8]:
from openmm.app import *
from openmm import *
from openmm.unit import *

pdb = PDBFile('/data/numerik/ag_cmd/arostami/restriants/mor_membrane_minimized.pdb')
forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')
system = forcefield.createSystem(pdb.topology, nonbondedMethod=PME,
                                 nonbondedCutoff=1*nanometer, constraints=HBonds)

In [14]:
restraint = CustomExternalForce('k*periodicdistance(x, y, z, x0, y0, z0)^2')
system.addForce(restraint)
restraint.addGlobalParameter('k', 100.0*kilojoules_per_mole/nanometer**2)
restraint.addPerParticleParameter('x0')
restraint.addPerParticleParameter('y0')
restraint.addPerParticleParameter('z0')

2

In [15]:
for atom in pdb.topology.atoms():
    if atom.name == 'POP':
        restraint.addParticle(atom.index, pdb.positions[atom.index])

In [16]:
integrator = LangevinMiddleIntegrator(300*kelvin, 1/picosecond, 0.004*picoseconds)
simulation = Simulation(pdb.topology, system, integrator)
simulation.context.setPositions(pdb.positions)
simulation.step(10)

In [17]:
simulation.reporters.append(PDBReporter('/data/numerik/ag_cmd/arostami/restriants/test1.pdb', 1000))
simulation.reporters.append(StateDataReporter(stdout, 1000, step=True,
                            potentialEnergy=True, temperature=True))
# run simulation
simulation.step(10000)

#"Step","Potential Energy (kJ/mole)","Temperature (K)"
1000,-552226.3966796035,292.4466851498266
2000,-549317.2806928366,300.54656102702353
3000,-551043.1760443293,302.56776185175585
4000,-552380.8768958952,300.43770193313406
5000,-553999.1261122159,301.13374883261497
6000,-553904.8748682573,302.01031003333367
7000,-555071.383335933,298.86821962017626
8000,-556271.4913375585,302.35132652125543
9000,-556721.2209613819,300.75107930141905
10000,-557155.399246586,300.04075157803027


In [ ]:
/data/numerik/ag_cmd/arostami/restriants/test1.pdb